In [1]:
#загрузка нужных библиотек
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split 

In [2]:
# данные для обучения и проверки модели
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [105]:
# визуально оценим данные
train.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [4]:
# соединим стобцы title и description (разделяя их пробелом во избежании склеивания слов на границе) для дальнейшего удобства
# задания словаря и обучения
train_text = train['title'] + ' ' + train['description']
test_text = test['title'] + ' ' + test['description']

На данным этапе применим алгоритм TfidfVectorizer() для преобразования нашего текствого значения стобцов title и description в sparse матрицу, где каждый стобец соответствует слову нашего "словаря", а строка соотвествует номеру строчки объявления из данных train. Каждая ячейка содержит число, которое характеризуется следуюшими позициями: кол-во раз, которое слово встретилось в строке умноженное на обратную пропорциональность встречаемости в разных текстах. Это и есть суть метода TF-IDF. Получили своеобразный "мешок слов".
Относительно специфики применения: проанализиуем только слова

In [5]:
word_vectorizer = TfidfVectorizer(sublinear_tf=True,analyzer='word',lowercase=True)
word_vectorizer.fit(train_text)
train_word_features = word_vectorizer.fit_transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [6]:
# выделим целевую переменную
train_target = train['category_id']

In [7]:
train_word_features

<489517x508666 sparse matrix of type '<class 'numpy.float64'>'
	with 15745105 stored elements in Compressed Sparse Row format>

In [8]:
# разделим наш train на обучающую и тестовую выборки
X_train, X_test,y_train, y_test=train_test_split(train_word_features,train_target, test_size= 0.33, random_state=241)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((342661, 508666), (146856, 508666), (342661,), (146856,))

In [13]:
# Обучим модель наивного байесовского классификатора на текстовых данных и получим accuracy для всей модели
clf2 = MultinomialNB().fit(X_train, y_train)
pred2 = clf2.predict(X_test)
score2 = accuracy_score(y_test,pred2)
score2

0.8586642697608542

неплохой результат для начала

In [11]:
# Теперь для классификации попробуем применить метод опорных векторов 
clf_svm = LinearSVC()
clf_svm.fit(X_train,y_train)
y_pred_svm = clf_svm.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred_svm)

0.888101269270578

Результат уже лучше, почти 89%. Дальше попробуем добавить к обучающей выборки на цены (стобец price).

In [106]:
# создадим новые переменные и получим ту же sparse матрицу
ngram_counter = TfidfVectorizer()
X_train0 = ngram_counter.fit_transform(train_text)
X_test0 = ngram_counter.transform(test_text)

# разделим эту выборку train на обучающую и тестовую, чтобы не портить основную
X_train, X_test,y_train, y_test=train_test_split(X_train0,train['category_id'], test_size= 0.33, random_state=241)

In [16]:
# стоблец цен преобразуем в элемент sparse и транспонируем
from scipy import sparse
mat_coo = sparse.coo_matrix(train['price'])
mat_test = sparse.coo_matrix(test['price'])

In [17]:
mat_coo = mat_coo.transpose()
mat_test = mat_test.transpose()

In [18]:
X_train0

<489517x508666 sparse matrix of type '<class 'numpy.float64'>'
	with 15745105 stored elements in Compressed Sparse Row format>

In [19]:
# добавим этот столбец цен к нашей матрице с "мешком слов"
from scipy import sparse
X_train_new=hstack((mat_coo,X_train0))
X_test_new=hstack((mat_test,X_test0))

In [20]:
X_train_new

<489517x508667 sparse matrix of type '<class 'numpy.float64'>'
	with 16234622 stored elements in COOrdinate format>

In [72]:
# и обучим модель методом опорных веторов (поскольку он выдал лучшее пока значение accuracy)
clf_svm1 = LinearSVC()
model1 = clf_svm1.fit(X_train, y_train)

In [73]:
# протестируем
y_pred1 = model1.predict(X_test)
score1= accuracy_score(y_test, y_pred1)
score1

0.5278708394617857

Получили не очень хорошее значение, столбец цен сильно испортил классификатор, к тому же из-за добавления заполненнго вектора к sparse матрице, обучение классификатора заняло ~1,5-2 часа. далее перейдем к оценке accuracy.

In [47]:
# для этого возьмем наилучший классификатор(метод опорных вектороры, он же svm) и представим наши данные в удобном виде
y_pred_er=[y_pred_svm[i] for i in range(len(y_pred_svm))]   # вектор предстаказанных моделью значений
y_test_er=[y_test.iloc[i] for i in range(len(y_test))]      # вектор тестирования, где указана правильная классификация

Инициализируем векторы правильных и неправильных значений

In [95]:
right=[0 for i in range(54)]
wrong=[0 for i in range(54)]

Пройдемся по нашей выборкеи заполним вектора правильных/неправильных значений

In [97]:
for i in range(len(y_pred_er)):
    if y_pred_er[i] == y_test_er[i]:
        right[y_test_er[i]] += 1
    else:
        wrong[y_test_er[i]] += 1

In [103]:
#выведем accuracy для каждого класса
accur=[0 for i in range(54)]
for i in range(len(accur)):
    accur[i]=right[i]/(right[i]+wrong[i])
# где позиция в векторе accur ставится в соответствие классу

In [116]:
accur_for_each= pd.DataFrame(data=accur, columns=['accuracy категории'])
accur_for_each.head(10)

,accuracy категории
0,0.918816
1,0.965844
2,0.914029
3,0.952964
4,0.926744
5,0.964023
6,0.823302
7,0.967207
8,0.938958
9,0.870184


Далее посчитаем полувручную базовые иерархии (Бытовая электроника, личные вещи и тд)

In [124]:
accur00=sum(right[0:15])/(sum(right[0:15])+sum(wrong[0:15]))
accur10=sum(right[15:30])/(sum(right[15:30])+sum(wrong[15:30]))
accur20=sum(right[30:42])/(sum(right[30:42])+sum(wrong[30:42]))
accur30=sum(right[42:])/(sum(right[42:])+sum(wrong[42:]))

In [129]:
accur_for_1ue = {'Бытовая электроника':[accur00],'Для дома и дачи':[accur10],'Личные вещи':[accur20],'Хобби и отдых':[accur30],}
accur_for_1ue = pd.DataFrame(data=accur_for_1ue)
accur_for_1ue

,Бытовая электроника,Для дома и дачи,Личные вещи,Хобби и отдых
0,0.934391,0.869346,0.87189,0.872659


Выше представлен вывод accuracy для высшей ступени иерархии. Подобным образом можно составить вывод accuracy для каждого уровня иерархии.

In [132]:
# результат скоринга файла test.csv с помощью классификатора smv
itog = pd.DataFrame() 
itog['item_id']= test['item_id']
itog['category_id'] = clf_svm.predict(test_word_features)
itog.to_csv('result.csv', index=False)